#### For this kaggle project, I wanted to utilize a logistic regression model to predict weather a passanger on the titanic would survive based on several factors. I was given a train (set to train the model) and test data set (set to predict). The test dataset included the same columns as the train dataset but did not include weather that individual survived our not.

#### Import libraries

In [ ]:
import pandas as pd
import numpy as np

Importing datasets and data cleaning

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test1 = pd.read_csv('test.csv')
test_ids = test1['PassengerId']

In [ ]:
train = train[['Sex', 'Age','Pclass', 'SibSp',
       'Parch', 'Fare', 'Embarked', 'Survived']]

test = test1[['Sex', 'Age','Pclass', 'SibSp',
       'Parch', 'Fare', 'Embarked']]

train.head()

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
0,male,22.0,3,1,0,7.2500,S,0
1,female,38.0,1,1,0,71.2833,C,1
2,female,26.0,3,0,0,7.9250,S,1
3,female,35.0,1,1,0,53.1000,S,1
4,male,35.0,3,0,0,8.0500,S,0


In [ ]:
test['Age'].fillna((test['Age']).mean(),inplace = True)
test['Fare'].fillna((test['Fare']).mean(),inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
test.head()

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked
0,male,34.5,3,0,0,7.8292,Q
1,female,47.0,3,1,0,7.0000,S
2,male,62.0,2,0,0,9.6875,Q
3,male,27.0,3,0,0,8.6625,S
4,female,22.0,3,1,1,12.2875,S


In [ ]:
train.head(
)

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
0,male,22.0,3,1,0,7.2500,S,0
1,female,38.0,1,1,0,71.2833,C,1
2,female,26.0,3,0,0,7.9250,S,1
3,female,35.0,1,1,0,53.1000,S,1
4,male,35.0,3,0,0,8.0500,S,0


In [ ]:
train.columns

Index(['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked',
       'Survived'],
      dtype='object')

In [ ]:
train.isna().sum()

Sex           0
Age         177
Pclass        0
SibSp         0
Parch         0
Fare          0
Embarked      2
Survived      0
dtype: int64

In [ ]:
train = train.dropna()

In [ ]:
train

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
0,male,22.0,3,1,0,7.2500,S,0
1,female,38.0,1,1,0,71.2833,C,1
2,female,26.0,3,0,0,7.9250,S,1
3,female,35.0,1,1,0,53.1000,S,1
4,male,35.0,3,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...
885,female,39.0,3,0,5,29.1250,Q,0
886,male,27.0,2,0,0,13.0000,S,0
887,female,19.0,1,0,0,30.0000,S,1
889,male,26.0,1,0,0,30.0000,C,1


#### Logistic regression models can only work if all of the columns you are testing have numerical indexes. Two of my columns have string values (Sex and Survived). Preprocessing.LabelEncoder() from sklearn will assign unique indexes (in this case just 0 and 1) for each unique value within the Sex and Survived column in my datasets.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ['Sex','Embarked']

for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])
  print(le.classes_)

train.head()

['female' 'male']
['C' 'Q' 'S']


<ipython-input-84-0ef4bda4013a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col] = le.transform(test[col])


,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
0,1,22.0,3,1,0,7.2500,2,0
1,0,38.0,1,1,0,71.2833,0,1
2,0,26.0,3,0,0,7.9250,2,1
3,0,35.0,1,1,0,53.1000,2,1
4,1,35.0,3,0,0,8.0500,2,0


In [ ]:
train.corr()

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
Sex,1.000000,0.099037,0.150826,-0.106296,-0.249543,-0.182457,0.109639,-0.536762
Age,0.099037,1.000000,-0.365902,-0.307351,-0.187896,0.093143,-0.032565,-0.082446
Pclass,0.150826,-0.365902,1.000000,0.065187,0.023666,-0.552893,0.244145,-0.356462
SibSp,-0.106296,-0.307351,0.065187,1.000000,0.383338,0.139860,0.033064,-0.015523
Parch,-0.249543,-0.187896,0.023666,0.383338,1.000000,0.206624,0.011803,0.095265
Fare,-0.182457,0.093143,-0.552893,0.139860,0.206624,1.000000,-0.283510,0.266100
Embarked,0.109639,-0.032565,0.244145,0.033064,0.011803,-0.283510,1.000000,-0.181979
Survived,-0.536762,-0.082446,-0.356462,-0.015523,0.095265,0.266100,-0.181979,1.000000


In [ ]:
train.head()

,Sex,Age,Pclass,SibSp,Parch,Fare,Embarked,Survived
0,1,22.0,3,1,0,7.2500,2,0
1,0,38.0,1,1,0,71.2833,0,1
2,0,26.0,3,0,0,7.9250,2,1
3,0,35.0,1,1,0,53.1000,2,1
4,1,35.0,3,0,0,8.0500,2,0


#### Now that all of the data is cleaned, it's time to use logistic regression to predict the "Survived" column of our test dataset.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train['Survived']
X = train.drop('Survived', axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=50)

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=500).fit(X_train,y_train)

In [ ]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8411214953271028

#### The above code shows that when the model is applied to 30% of my "train" dataset, it was correct 84% of the time indicating solid model performance. 

In [ ]:
prediction = clf.predict(test)
prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [ ]:
df = pd.DataFrame({'PassengerId':test_ids.values,
                   'Survived': prediction})

In [ ]:
df.to_csv('submit1.csv',index = False)


## After submitting the csv to Kaggle, I found that my model compared the correct 'Survived' value for 76% of the dataset. To better improve the model, you could include additional variables such as "title" within the name field.

## Overall this was one of my best model performances to date! In the future, as I learn more about deep learning, I hope to include those concepts to see if my accuracy score increases.